In [1]:
import pandas as pd
import json
import pathlib

In [2]:
papers = pd.DataFrame()

Title and abstract [_]

Document classification codes

Date of publication

Affiliations

Citations / references

Keywords

In [3]:
data_list = []

In [ ]:
for path in pathlib.Path("Data 2018-2023/Project").glob("*/*"):
    try:
        with open(path) as file:
            data = json.load(file)
        filtered_data = {}
        filtered_data['title'] = data['abstracts-retrieval-response']['coredata']['dc:title']
        filtered_data['abstract'] = data['abstracts-retrieval-response']['item']['bibrecord']['head']['abstracts']
        filtered_data['date'] = data['abstracts-retrieval-response']['coredata']['prism:coverDate']
        filtered_data['cited_by_count'] = data['abstracts-retrieval-response']['coredata']['citedby-count']
        filtered_data['reference_count'] = data['abstracts-retrieval-response']['item']['bibrecord']["tail"]["bibliography"]["@refcount"]
        data_list.append(filtered_data)
        print(f"Success: {path}")
    except Exception as e:
        print(f"Failed: {path}\nException: {e}")
    finally:
        print()

Success: Data 2018-2023\Project\2018\201800000

Success: Data 2018-2023\Project\2018\201800001

Success: Data 2018-2023\Project\2018\201800002

Success: Data 2018-2023\Project\2018\201800003

Success: Data 2018-2023\Project\2018\201800004

Success: Data 2018-2023\Project\2018\201800005

Success: Data 2018-2023\Project\2018\201800006

Success: Data 2018-2023\Project\2018\201800007

Success: Data 2018-2023\Project\2018\201800008

Success: Data 2018-2023\Project\2018\201800009

Success: Data 2018-2023\Project\2018\201800010

Success: Data 2018-2023\Project\2018\201800011

Success: Data 2018-2023\Project\2018\201800012

Success: Data 2018-2023\Project\2018\201800013

Success: Data 2018-2023\Project\2018\201800014

Success: Data 2018-2023\Project\2018\201800015

Success: Data 2018-2023\Project\2018\201800016

Success: Data 2018-2023\Project\2018\201800017

Success: Data 2018-2023\Project\2018\201800018

Success: Data 2018-2023\Project\2018\201800019

Success: Data 2018-2023\Project\2018\201